In [1]:
import random
import numpy as np
from sklearn.preprocessing import StandardScaler
import function

In [3]:
X_1,X_2,y=function.gene_data(p_1=50,p_2=50,n=100,r_J=10,r_1=5,r_2=5,r_prop=0.8,w_J=2.0,w_1=1.0,w_2=1.0,X1_erro=0.2,X2_erro=0.2,y_erro=0.2)